### Import Divvy Bike Data: 

mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202201-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202202-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202203-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202204-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202205-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202206-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202207-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202208-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202209-divvy-publictripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202210-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202211-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline  202212-divvy-tripdata.csv<br>

### Import Weather Data: 
mongoimport --type csv -d chicago_bikes -c weather_daily --headerline weather_daily.csv

In [60]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json

# Adding for query to find top ten stations 
#from pymongo.collection import Collection
#from pymongo.aggregation import Aggregation

In [61]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [62]:
# check our list of collections
print(mongo.list_database_names())

['admin', 'chicago_bikes', 'config', 'local']


In [63]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [88]:
# review the collections in our new database
print(db.list_collection_names())

['divvy_rides_by_season', 'weather_daily', 'withoutStationName', 'divvy_ridedata_merged', 'sig_prcp_no', 'distinct_station_names', 'Top10Routes', 'Top10EndStations', 'Top10StartStations', 'sig_prcp_yes', 'RouteDistance', 'withStationName', 'divvy_rides_by_month', 'withLatLong', 'divvy_ridedata']


In [65]:
# review a document in the customer_list collection
print(db.divvy_ridedata.find_one())

{'_id': ObjectId('655f1914044b5b5ebbf90d67'), 'ride_id': 'A6CF8980A652D272', 'rideable_type': 'electric_bike', 'started_at': '2022-01-10 08:41:56', 'ended_at': '2022-01-10 08:46:17', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.012763, 'start_lng': -87.6659675, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual'}


In [66]:
divvy_rides = db['divvy_ridedata']
weather_daily = db['weather_daily']
withStation = db['withStationName']
withoutStation = db['withoutStationName']

In [67]:
print(divvy_rides.count_documents({}))
print(weather_daily.count_documents({}))

5667717
365


In [68]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["started_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "started_at_date": date_part,
                    "started_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)


In [69]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["ended_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "ended_at_date": date_part,
                    "ended_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)

In [70]:
print(db.divvy_ridedata.find_one())
print(db.weather_daily.find_one())

{'_id': ObjectId('655f1914044b5b5ebbf90d67'), 'ride_id': 'A6CF8980A652D272', 'rideable_type': 'electric_bike', 'started_at': '2022-01-10 08:41:56', 'ended_at': '2022-01-10 08:46:17', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.012763, 'start_lng': -87.6659675, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual', 'started_at_date': '2022-01-10', 'started_at_time': '08:41:56', 'ended_at_date': '2022-01-10', 'ended_at_time': '08:46:17'}
{'_id': ObjectId('655f19d5f1c6af1dd0f57470'), 'date': '2022-01-01', 'cloud_cover': 90.0, 'precipitation': 0.18, 'min_temp': 33.22, 'max_temp': 42.1, 'morning_temp': 42.1, 'afternoon_temp': 38.43, 'evening_temp': 35.24, 'night_temp': 38.44, 'max_windspeed': 15.01}


In [71]:
divvy_rides.create_index([("started_at_date", 1)])
weather_daily.create_index([("date", 1)])


'date_1'

In [92]:
pipeline = [
    {
        '$lookup': {
            'from': 'weather_daily',
            'localField': 'started_at_date',
            'foreignField': 'date',
            'as': 'weather_data'
        }
    },
    {
        '$unwind': {
            'path': '$weather_data',
            'preserveNullAndEmptyArrays': True
        }
    },
    {
        '$merge': {
            'into': 'divvy_ridedata_merged',  # Replace with your new collection name
            'whenMatched': 'merge',  # Merge documents with matching _id fields
            'whenNotMatched': 'insert'  # Insert documents that don't match
        }
    },
]

divvy_rides.aggregate(pipeline)
print("Update completed successfully.")


Update completed successfully.


In [93]:
# Access the database and collection
collection = db["divvy_ridedata_merged"]  

# Count the documents in the collection
document_count = collection.count_documents({})
print("Total number of documents:", document_count)


Total number of documents: 5667717


In [94]:
divvy_ridedata_merged = db['divvy_ridedata_merged']
divvy_ridedata_merged.find_one()

{'_id': ObjectId('655f1914044b5b5ebbf90d67'),
 'end_lat': 42.01256011541,
 'end_lng': -87.6743671152,
 'end_station_id': 'RP-007',
 'end_station_name': 'Clark St & Touhy Ave',
 'ended_at': '2022-01-10 08:46:17',
 'ended_at_date': '2022-01-10',
 'ended_at_time': '08:46:17',
 'member_casual': 'casual',
 'ride_id': 'A6CF8980A652D272',
 'rideable_type': 'electric_bike',
 'start_lat': 42.012763,
 'start_lng': -87.6659675,
 'start_station_id': 525,
 'start_station_name': 'Glenwood Ave & Touhy Ave',
 'started_at': '2022-01-10 08:41:56',
 'started_at_date': '2022-01-10',
 'started_at_time': '08:41:56',
 'weather_data': {'_id': ObjectId('655f19d5f1c6af1dd0f5747e'),
  'date': '2022-01-10',
  'cloud_cover': 17.0,
  'precipitation': 0.24,
  'min_temp': 9.5,
  'max_temp': 20.84,
  'morning_temp': 13.69,
  'afternoon_temp': 9.73,
  'evening_temp': 14.0,
  'night_temp': 20.75,
  'max_windspeed': 16.35,
  'prcp_inches': 0.0094488,
  'sig_prcp': 'no',
  'avg_temp': 12.473333333333334}}

In [89]:
# review the collections in our new database
print(db.list_collection_names())

['divvy_rides_by_season', 'weather_daily', 'withoutStationName', 'divvy_ridedata_merged', 'sig_prcp_no', 'distinct_station_names', 'Top10Routes', 'Top10EndStations', 'Top10StartStations', 'sig_prcp_yes', 'RouteDistance', 'withStationName', 'divvy_rides_by_month', 'withLatLong', 'divvy_ridedata']


In [76]:
# Use aggregation pipeline to create a collection that contains start and end station names
pipeline = [
         {"$match": {"start_station_name": {"$exists": True, "$ne": ""}, 
                     "end_station_name":{"$exists": True, "$ne": ""}}},

         {"$out": "withStationName"}
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))


In [77]:
#check to make sure that collections is updated
db.list_collection_names()

['weather_daily', 'divvy_ridedata_merged', 'withStationName', 'divvy_ridedata']

In [78]:
#check if there are any documents without start or end station names
print(db.withStationName.find_one({"start_station_name":""}))
print(db.withStationName.find_one({"end_station_name":""}))

None
None


In [79]:
# Use aggregation pipeline to create a collection that doesn't contain start and end station names
pipeline = [
         {"$match": {"start_station_name": {"$exists": True, "$eq": ""}, 
                     "end_station_name":{"$exists": True, "$eq": ""}}},

         {"$out": "withoutStationName"}
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))

In [80]:
db.list_collection_names()

['weather_daily',
 'withoutStationName',
 'divvy_ridedata_merged',
 'withStationName',
 'divvy_ridedata']

In [81]:
#check if there are any documents without start or end station names
print(db.withoutStationName.find_one({"start_station_name":{"$ne":""}}))
print(db.withoutStationName.find_one({"end_station_name":{"$ne":""}}))

None
None


In [82]:
# Use aggregation pipeline to find top ten start stations 
pipeline = [
    {
        "$group": {
            "_id": "$start_station_name",
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 10
    },
    {   "$out": "Top10StartStations"
}
]

# Perform the aggregation
result = list(withStation.aggregate(pipeline))

# Assign collection to a variable
Top10StartStations = db['Top10StartStations']

In [83]:
# Check to see that the collection was added
db.list_collection_names()

['weather_daily',
 'withoutStationName',
 'divvy_ridedata_merged',
 'Top10StartStations',
 'withStationName',
 'divvy_ridedata']

In [84]:
Top10StartStations.find_one()

{'_id': 'Streeter Dr & Grand Ave',
 'count': 71269,
 'latitude': 41.880958,
 'longitude': -87.616743}

In [85]:
# Use aggregation pipeline to find top ten end stations 
pipeline = [
    {
        "$group": {
            "_id": "$end_station_name",
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 10
    },
    {   "$out": "Top10EndStations"
}
]

# Perform the aggregation
result = list(withStation.aggregate(pipeline))

# Assign to a variable
Top10EndStations = db['Top10EndStations']

In [86]:
# Create a pipeline query to find the top ten bike routes (by start and end station)
pipeline = [
    {
        "$group": {
            "_id": { "Start Station": "$start_station_name", "End Station": "$end_station_name"},
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {"$sort": {"count": -1}
},
    {
        "$limit": 10
},
    {   "$out": "Top10Routes"
}
]
# Perform the aggregation
result = list(withStation.aggregate(pipeline))

# Assign to a variable
Top10Routes = db['Top10Routes']

In [ ]:
# Close the MongoDB connection
mongo.close()